# Machine Learning Task 1
## Silvana Belegu
### Task Description
Implement parallel programming techniques for running KNN and perform benchmark studies 

In [4]:
import multiprocessing as mp

In [5]:
print("Number of processors: ", mp.cpu_count())

Number of processors:  128


In [2]:
from KNNClassifierP import KNNClassifierP
import numpy as np

In [3]:
rows = 100000
cols = 500
np.random.seed(699)
X_train = np.random.rand(rows*cols).reshape((rows,cols))
y_train = np.random.randint(2, size=rows)
print(f'X_train shape {X_train.shape} - y_train shape {y_train.shape}')

X_train shape (100000, 500) - y_train shape (100000,)


Running the code sequentially 

In [22]:
from KNNClassifier import KNNClassifier
import numpy as np
import time, timeit
import cProfile, pstats
from tqdm.notebook import tqdm

In [18]:
rows = 100000
cols = 500
np.random.seed(699)
X_train = np.random.rand(rows*cols).reshape((rows,cols))
y_train = np.random.randint(2, size=rows)
print(f'X_train shape {X_train.shape} - y_train shape {y_train.shape}')

X_train shape (100000, 500) - y_train shape (100000,)


In [19]:
knn = KNNClassifier(k=2)
knn.fit(X_train, y_train)

In [20]:
test_size = 1000
X_test = np.random.randint(rows, size=test_size)

In [21]:
cProfile.run('knn.predict(X_train[X_test])', 'profile_results.prof')

In [25]:
stats = pstats.Stats('profile_results.prof')
stats.sort_stats('cumulative').print_stats(30)

Sun Nov 24 21:53:20 2024    profile_results.prof

         1000017006 function calls in 911.125 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  911.125  911.125 {built-in method builtins.exec}
        1    0.001    0.001  911.125  911.125 <string>:1(<module>)
        1    0.000    0.000  911.124  911.124 /mnt/lscratch/users/sbelegu/parallel_programming/KNNClassifier.py:17(predict)
        1    0.489    0.489  911.124  911.124 /mnt/lscratch/users/sbelegu/parallel_programming/KNNClassifier.py:18(<listcomp>)
     1000    0.011    0.000  910.635    0.911 /mnt/lscratch/users/sbelegu/parallel_programming/KNNClassifier.py:21(_predict)
     1000   55.938    0.056  897.260    0.897 /mnt/lscratch/users/sbelegu/parallel_programming/KNNClassifier.py:23(<listcomp>)
100000000  332.990    0.000  841.322    0.000 /mnt/lscratch/users/sbelegu/parallel_programming/KNNClassifier.py:11(euclidean_distance)
10000000

In [26]:
from KNNClassifier_parallel import KNNClassifier_parallel

In [27]:
knn_parallel = KNNClassifier_parallel(k=2, num_workers= 32)

In [28]:
%%timeit
knn_p = knn_parallel.predict(X_train[X_test])

AttributeError: 'KNNClassifier_parallel' object has no attribute 'X_train'

To make the `KNNClassifier` class faster, I focused on parallelizing the most time-consuming part of the algorithm: calculating the distances and predicting the labels. The reason this step is so costly is that, for each query point, we need to calculate the distance to all the training points. These calculations are independent of one another, which makes them ideal for parallelization.

To speed things up, I'll use multi-processing with Python's `concurrent.futures.ProcessPoolExecutor`. This approach is particularly effective for CPU-bound tasks, like the distance calculations in k-NN, because it allows us to take advantage of multiple CPU cores. By doing this, we can bypass Python's Global Interpreter Lock (GIL), which typically limits the performance of multi-threading in CPU-heavy tasks.

In [ ]:
# Function to benchmark k-NN
def benchmark_knn(knn_function, num_runs=30, num_workers=4):
    real_times = []
    cpu_times = []
    
    for _ in range(num_runs):
        # Measure real time
        start_real = time.time()
        start_cpu = time.process_time()
        
        if knn_function == knn_parallel:
            knn_function(X_train, y_train, X_query, k=k, num_workers=num_workers)
        else:
            knn_function(X_train, y_train, X_query, k=k)
        
        end_real = time.time()
        end_cpu = time.process_time()
        
        real_times.append(end_real - start_real)
        cpu_times.append(end_cpu - start_cpu)
    
    avg_real_time = np.mean(real_times)
    std_real_time = np.std(real_times)
    avg_cpu_time = np.mean(cpu_times)
    std_cpu_time = np.std(cpu_times)
    
    return avg_real_time, std_real_time, avg_cpu_time, std_cpu_time

# Benchmark both the sequential and parallel versions
sequential_results = benchmark_knn(knn_sequential, X_train, y_train, X_query)
parallel_results = benchmark_knn(knn_parallel, X_train, y_train, X_query, num_workers=4)

# Print the results
print(f"Sequential k-NN - Avg Real Time: {sequential_results[0]:.4f}s, Std Real Time: {sequential_results[1]:.4f}s")
print(f"Sequential k-NN - Avg CPU Time: {sequential_results[2]:.4f}s, Std CPU Time: {sequential_results[3]:.4f}s")

print(f"Parallel k-NN - Avg Real Time: {parallel_results[0]:.4f}s, Std Real Time: {parallel_results[1]:.4f}s")
print(f"Parallel k-NN - Avg CPU Time: {parallel_results[2]:.4f}s, Std CPU Time: {parallel_results[3]:.4f}s")


In [10]:
start_real = time.time()
start_cpu = time.process_time()

predictions = knn.predict(X_train[X_test])

end_real = time.time()
end_cpu = time.process_time()
real_times.append(end_real-start_real)
cpu_times.append(end_cpu - start_cpu)

KeyboardInterrupt: 

In [ ]:
print(f'correct {np.sum(y_train[X_test] == predictions)}')

In [ ]:
import time

In [13]:
num_run=30

In [ ]:
real_times = []
cpu_times =[]

for _ in range(num_run):
    start_real = time.time()
    start_cpu = time.process_time()

    predictions = knn.predict(X_train[X_test])
    print(f'correct {np.sum(y_train[X_test] == predictions)}')

    end_real = time.time()
    end_cpu = time.process_time()
    real_times.append(end_real-start_real)
    cpu_times.append(end_cpu - start_cpu)



correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743
correct 743


In [ ]:
real_time_avg = np.mean(real_times)
real_time_std = np.std(real_times)
cpu_time_avg = np.mean(cpu_times)
cpu_time_std = np.std(cpu_times)

In [ ]:
print(f"Real time: Avg = {real_time_avg:.6f}s, Std Dev = {real_time_std:.6f}s")
print(f"Real time: Avg = {cpu_time_avg:.6f}s, Std Dev = {cpu_time_std:.6f}s")

In [ ]:
import matplotlib.pyplot as plt

plt.hist(real_times, bins=10, alpha=0.5, label='Real Time')
plt.hist(cpu_times, bins=10, alpha=0.5, label='CPU Time')
plt.legend()
plt.title('Benchmark Time Distribution')
plt.xlabel('Time (s)')
plt.ylabel('Frequency')
plt.show()
